In [18]:
import pandas as pd
import numpy as np

league_2017_ml = pd.read_csv('csv/nhl.20162017/neural_model20162017.csv')
league_2017_ml.head()

,Team,Name,Playoffs?,Conference,Division,Season End,Season Type,Game,Points,Corsi Close,...,B Agnst Down 1,B% Tied,B For Tied,B Agnst Tied,B% Up 1,B For Up 1,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2
0,ANA,Ducks,Playoff,Western,Pacific,2016,Regular,82,103,53.68,...,103,52.45,385,349,48.63,195,206,33.97,71,138
1,ARI,Coyotes,DNQ,Western,Pacific,2016,Regular,82,78,45.65,...,160,45.38,295,355,43.02,154,204,40.68,48,70
2,BOS,Bruins,DNQ,Eastern,Atlantic,2016,Regular,82,93,49.87,...,133,51.31,392,372,41.82,161,224,37.89,72,118
3,BUF,Sabres,DNQ,Eastern,Atlantic,2016,Regular,82,81,47.02,...,205,47.49,340,376,41.22,115,164,36.89,45,77
4,CAR,Hurricanes,DNQ,Eastern,Metropolitan,2016,Regular,82,86,51.55,...,129,54.29,424,357,47.80,163,178,38.46,45,72


### apply one-hot encoding

In [ ]:
data  = league_2017_ml.values
Team = data[:,0]
Name = data[:,1]
Playoffs = data[:,2]
Conference = data[:,3]
Division = data[:,4 ]
SeasonType = data[:,6]

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [ ]:
label_encoder.fit(Team)
encoded_team = label_encoder.transform(Team)

In [ ]:
label_encoder.fit(Name)
encoded_name = label_encoder.transform(Name)

In [ ]:
label_encoder.fit(Playoffs)
encoded_playoffs = label_encoder.transform(Playoffs)

In [ ]:
label_encoder.fit(Conference)
encoded_conference = label_encoder.transform(Conference)

In [ ]:
label_encoder.fit(Division)
encoded_division = label_encoder.transform(Division)

In [ ]:
label_encoder.fit(SeasonType)
encoded_season = label_encoder.transform(SeasonType)

In [ ]:
concatenate = {'Team': encoded_team,
               'Name': encoded_name, 
               'Playoffs': encoded_playoffs,
               'Conference': encoded_conference,
               'Division': encoded_division,
               'SeasonType': encoded_season,
               }

encoding_df = pd.DataFrame(data = concatenate)


In [ ]:
encoding_df.head()

In [ ]:
# encoding_df.to_csv('csv/encoded_20162017.csv', index = False)

### Neural Network

In [3]:
encoded_ml = pd.read_csv('csv/nhl.20162017/nhl.encoded_20162017.csv')
encoded_ml.head()

,Conference,Division,Name,Playoffs,SeasonType,Team,Season End,Game,Points,Corsi Close,...,B Agnst Down 1,B% Tied,B For Tied,B Agnst Tied,B% Up 1,B For Up 1,B Agnst Up 1,B% Up 2,B For Up 2,B Agnst Up 2
0,1,3,10,1,0,0,2016,82,103,53.68,...,103,52.45,385,349,48.63,195,206,33.97,71,138
1,1,3,8,0,0,1,2016,82,78,45.65,...,160,45.38,295,355,43.02,154,204,40.68,48,70
2,0,0,4,0,0,2,2016,82,93,49.87,...,133,51.31,392,372,41.82,161,224,37.89,72,118
3,0,0,25,0,0,3,2016,82,81,47.02,...,205,47.49,340,376,41.22,115,164,36.89,45,77
4,0,2,13,0,0,4,2016,82,86,51.55,...,129,54.29,424,357,47.80,163,178,38.46,45,72


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

Using TensorFlow backend.


In [5]:
X = encoded_ml.drop('Playoffs', axis = 1)
y = encoded_ml['Playoffs']

In [6]:
X.shape

(4350, 71)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 1, stratify = y)

In [8]:
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [11]:
from keras.models import Sequential
from keras.layers import Dense

#Create model and add layers
model = Sequential()
model.add(Dense(units = 100, activation = 'relu', input_dim = 71))
model.add(Dense(units = 100, activation = 'relu'))
model.add(Dense(units = 2, activation = 'softmax'))

In [12]:
#compile and fit the model
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs = 10,
    shuffle = True,
    verbose = 2)

Epoch 1/10
 - 1s - loss: 0.2767 - acc: 0.8945
Epoch 2/10
 - 0s - loss: 0.0800 - acc: 0.9816
Epoch 3/10
 - 0s - loss: 0.0319 - acc: 0.9948
Epoch 4/10
 - 0s - loss: 0.0179 - acc: 0.9954
Epoch 5/10
 - 0s - loss: 0.0163 - acc: 0.9957
Epoch 6/10
 - 0s - loss: 0.0060 - acc: 0.9994
Epoch 7/10
 - 0s - loss: 0.0035 - acc: 0.9997
Epoch 8/10
 - 0s - loss: 0.0017 - acc: 1.0000
Epoch 9/10
 - 0s - loss: 9.8156e-04 - acc: 1.0000
Epoch 10/10
 - 0s - loss: 7.3095e-04 - acc: 1.0000


In [13]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled,
    y_test_categorical,
    verbose = 2)

print(f'Normal Neural Network| Loss: {model_loss}, Accuracy: {model_accuracy}')

Normal Neural Network| Loss: 0.027959979065652826, Accuracy: 0.9935661764705882


In [14]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

print(f'Predicted classes: {prediction_labels}')
print(f'Actual Labels: {list(y_test[:5])}')

Predicted classes: [1 1 0 1 0]
Actual Labels: [1, 1, 0, 1, 0]


### saving a trained model

In [15]:
#saving the trained model using the HDF5 binary format with the extension .h5

model.save('neural_model/neural_nhl.h5')

### evaluating the loaded model

In [16]:
#load the model

from keras.models import load_model
predict_team = load_model('neural_model/neural_nhl.h5')

In [17]:
model_loss, model_accuracy = predict_team.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.027959979065652826, Accuracy: 0.9935661764705882
